# 07 — Evaluation

Measure quality across three dimensions:
1. **Retrieval recall** — Does semantic search return the right provisions?
2. **Classification accuracy** — Does the classifier assign correct risk levels?
3. **Faithfulness** — Do answers cite real, verifiable articles?

**Prerequisites**: Run Notebooks 01-02 first (ChromaDB index must exist).

In [ ]:
import sys
sys.path.insert(0, "..")

from src.eval import retrieval_recall, classification_accuracy, faithfulness_check
from src.tools import init_tools, TOOL_SCHEMAS
from src.agent import run_agent
from src.embeddings import get_openai_client
import json

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

## 1. Retrieval Recall

For a set of queries, check if the expected provisions appear in the top-k results.

In [ ]:
retrieval_queries = [
    {
        "query": "Welche KI-Systeme sind verboten?",
        "expected_ids": ["art_5"],
        "label": "Verbotene KI (Art. 5)",
    },
    {
        "query": "Wie werden Hochrisiko-KI-Systeme klassifiziert?",
        "expected_ids": ["art_6"],
        "label": "Hochrisiko-Klassifizierung (Art. 6)",
    },
    {
        "query": "Welche Transparenzpflichten gelten für KI-Systeme?",
        "expected_ids": ["art_50"],
        "label": "Transparenz (Art. 50)",
    },
    {
        "query": "Anbieterpflichten für Hochrisiko-KI-Systeme",
        "expected_ids": ["art_16"],
        "label": "Anbieterpflichten (Art. 16)",
    },
    {
        "query": "Betreiberpflichten gemäß der KI-Verordnung",
        "expected_ids": ["art_26"],
        "label": "Betreiberpflichten (Art. 26)",
    },
    {
        "query": "Sanktionen und Bußgelder bei Nichteinhaltung",
        "expected_ids": ["art_99"],
        "label": "Bußgelder (Art. 99)",
    },
    {
        "query": "Definition eines Systems der künstlichen Intelligenz",
        "expected_ids": ["art_3"],
        "label": "Begriffsbestimmungen (Art. 3)",
    },
    {
        "query": "Anforderungen an das Risikomanagementsystem",
        "expected_ids": ["art_9"],
        "label": "Risikomanagement (Art. 9)",
    },
    {
        "query": "Pflichten für KI-Modelle mit allgemeinem Verwendungszweck",
        "expected_ids": ["art_53"],
        "label": "GPAI-Pflichten (Art. 53)",
    },
    {
        "query": "Regulierung biometrischer Identifizierungssysteme",
        "expected_ids": ["art_5"],
        "label": "Biometrie (Art. 5)",
    },
]

recall_results = retrieval_recall(retrieval_queries, chroma_path="../chroma_db", lang=LANG)

print(f"Retrieval Recall@8: {recall_results['recall']:.1%}")
print(f"\nErgebnisse pro Abfrage:")
for d in recall_results["details"]:
    status = "OK" if d["recall"] == 1.0 else "MISS"
    print(f"  [{status}] {d['label']}: recall={d['recall']:.0%} (verfehlt: {d['missed']})")

## 2. Classification Accuracy

Run the compliance classifier on test cases with known risk levels.

In [ ]:
CLASSIFIER_PROMPT = """Du bist ein Klassifikator für die EU-KI-Verordnung. Bestimme die Risikostufe.
Prüfe Art. 5 (verboten), Art. 6 + Anhang III (Hochrisiko), Art. 50 (Transparenz/begrenzt).
Nutze die Tools für den Verordnungstext. Antworte NUR mit einem JSON-Objekt:
{"risk_level": "unacceptable" | "high" | "limited" | "minimal", "reasoning": "kurze Begründung"}"""

classification_cases = [
    {
        "description": "Ein Social-Scoring-System, das von einer Regierung zur Bewertung der Vertrauenswürdigkeit von Bürgern eingesetzt wird und den Zugang zu öffentlichen Diensten basierend auf Sozialverhalten einschränkt.",
        "expected_risk": "unacceptable",
        "label": "Social Scoring (Regierung)",
    },
    {
        "description": "Ein KI-System zur Sichtung und Bewertung von Bewerbungen für Einstellungsentscheidungen in einem großen Unternehmen.",
        "expected_risk": "high",
        "label": "KI-Recruiting-Tool",
    },
    {
        "description": "Ein Chatbot auf einer E-Commerce-Website, der Kunden bei der Produktsuche hilft und FAQs beantwortet. Er kennzeichnet sich eindeutig als KI.",
        "expected_risk": "limited",
        "label": "E-Commerce-Chatbot",
    },
    {
        "description": "Ein KI-basierter Spam-Filter für E-Mails, der eingehende Nachrichten automatisch kategorisiert.",
        "expected_risk": "minimal",
        "label": "E-Mail-Spam-Filter",
    },
    {
        "description": "Ein KI-System zur Bewertung der Kreditwürdigkeit von Personen, die Bankkredite beantragen.",
        "expected_risk": "high",
        "label": "Kreditscoring-KI",
    },
]


def classify_for_eval(description: str) -> str:
    """Run classifier and extract risk level."""
    response, _ = run_agent(
        client=client,
        system_prompt=CLASSIFIER_PROMPT,
        user_message=f"Klassifiziere: {description}",
        tools=TOOL_SCHEMAS,
    )
    try:
        start = response.index("{")
        end = response.rindex("}") + 1
        data = json.loads(response[start:end])
        return data.get("risk_level", "unknown")
    except (ValueError, json.JSONDecodeError):
        return "unknown"


print("Klassifizierungs-Evaluation läuft (LLM-Aufruf pro Testfall)...\n")
class_results = classification_accuracy(classification_cases, classify_for_eval)

print(f"Klassifizierungsgenauigkeit: {class_results['accuracy']:.0%}")
print(f"\nErgebnisse pro Testfall:")
for d in class_results["details"]:
    status = "OK" if d["correct"] else "FALSCH"
    print(f"  [{status}] {d['label']}: erwartet={d['expected']}, vorhergesagt={d['predicted']}")

## 3. Faithfulness Check

Verify that cited articles in agent responses actually exist in the regulation.

In [ ]:
# Zitierte Artikel verifizieren — existieren sie im Index?
faithfulness_cases = [
    {
        "answer": "Verbotene Praktiken sind in Art. 5 definiert",
        "cited_articles": ["art_5"],
    },
    {
        "answer": "Hochrisiko-Klassifizierung in Art. 6, Anhang III listet spezifische Bereiche",
        "cited_articles": ["art_6", "anx_III"],
    },
    {
        "answer": "Anbieterpflichten in Art. 16, Risikomanagement in Art. 9, Daten-Governance in Art. 10",
        "cited_articles": ["art_16", "art_9", "art_10"],
    },
    {
        "answer": "Transparenzpflichten in Art. 50, Betreiberpflichten in Art. 26",
        "cited_articles": ["art_50", "art_26"],
    },
    {
        "answer": "Bußgelder bis 35 Mio. EUR oder 7% des Umsatzes gemäß Art. 99",
        "cited_articles": ["art_99"],
    },
]

faith_results = faithfulness_check(faithfulness_cases, chroma_path="../chroma_db", lang=LANG)

print(f"Zitat-Treue: {faith_results['faithfulness']:.0%}")
print(f"\nDetails:")
for d in faith_results["details"]:
    status = "OK" if d["faithfulness"] == 1.0 else "PROBLEM"
    print(f"  [{status}] Zitiert: {d['cited']} → Verifiziert: {d['verified']}, Nicht verifiziert: {d['unverified']}")

## Zusammenfassung

In [ ]:
import matplotlib.pyplot as plt

metrics = {
    "Retrieval\nRecall@8": recall_results["recall"],
    "Klassifizierungs-\ngenauigkeit": class_results["accuracy"],
    "Zitat-\nTreue": faith_results["faithfulness"],
}

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(metrics.keys(), metrics.values(), color=["#2196F3", "#4CAF50", "#FF9800"], edgecolor="black")
ax.set_ylim(0, 1.1)
ax.set_ylabel("Score")
ax.set_title("EU-KI-Verordnung Agenten — Evaluationsmetriken")

for bar, val in zip(bars, metrics.values()):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
            f"{val:.0%}", ha="center", va="bottom", fontweight="bold")

plt.tight_layout()
plt.show()